# Organizando sua análise de dados com Snakemake

## Ciências de dados

- 10% análise
- 90% limpando dados, esperando rodar a análise...

## Um exemplo: bioinformática

- Várias ferramentas, executadas em sequência
- cada uma com diferentes tipos de entradas e saídas
    * e ferramentas extras pra converter formatos
- Reproducibilidade é importante
    * Mas...

## O seu pior inimigo é você mesmo, seis meses atrás
  
Soluções?
 - versionamento
 - documentação
 - automação

## shell scripts como documentação executável

- Rode tudo no shell
- `history > run.sh`
- É um começo...

## Melhor: um shell script comentado!

- Por que você está executando cada comando?

## Exemplo: 01-shell/run.sh

In [1]:
cd 01_shell

/home/chick/prj/2019-12-13-pyjamas/01_shell


In [2]:
!ls

inputs	outputs  run_improved.sh  run.sh


In [3]:
!cat run.sh

#! /bin/bash

# Baixa arquivo para classificação
mkdir -p inputs/HMP
wget -O inputs/HMP/HSMA33OT.fastq.gz https://ibdmdb.org/tunnel/static/HMP2/WGS/1750/HSMA33OT.fastq.gz

# Baixa base de dados para comparação
mkdir -p inputs/lca
wget -O inputs/lca/genbank-k21.lca.json.gz https://osf.io/d7rv8/download

# Calcula uma assinatura para o arquivo a ser classificado
mkdir -p outputs/2.3.0
sourmash compute -k 21,31,51 \
  --scaled 2000 \
  --track-abundance \
  --name-from-first \
  -o outputs/2.3.0/HSMA33OT.fastq.gz.sig \
  inputs/HMP/HSMA33OT.fastq.gz

# Classifica a assinatura contra a base de dados
mkdir -p outputs/2.3.0/lca_search/k21
sourmash search -o outputs/2.3.0/lca_search/k21/HSMA33OT.fastq.gz.log \
  --scaled 2000  \
  -k 21 \
  outputs/2.3.0/HSMA33OT.fastq.gz.sig \
  inputs/lca/genbank-k21.lca.json.gz


## Problemas com shell scripts

- Adicionando novos comandos? Reexecutar tudo...
    - Idempotência
- Qual a dependência entre comandos? Qual deve ser executado antes?
    - Fluxo de dados
- E se eu quiser executar com 100 entradas, em vez de uma só?
    - Paralelização

## Por que snakemake?

- Ideia similar ao Make:
    - alvos
    - dependências
- Escreva regras para gerar um arquivo
- snakemake se responsabiliza por encontrar a ordem de execução

## Exemplo: 02-snakemake/

In [4]:
cd ../02_snakemake/

/home/chick/prj/2019-12-13-pyjamas/02_snakemake
